### TF Record Generation

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import menpo.io as mio
from menpo.shape import PointCloud
from menpo.visualize import print_progress
from menpo.image import Image
from menpo.transform import Translation, Scale
import cv2

from pathlib import Path
import scipy.io as sio
from io import BytesIO

from scipy.spatial.distance import pdist
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
slim = tf.contrib.slim

import sys
import detect_face

from scipy import misc
from PIL import Image as PImage

In [4]:
def crop_image_bounding_box(img, bbox, res, base=256., order=1):
   
    center = bbox.centre()
    bmin, bmax = bbox.bounds()
    scale = np.linalg.norm(bmax - bmin) / base

    return crop_image(img, center, scale, res, base, order=order)

def crop_image(img, center, scale, res, base=256., order=1):
    h = scale

    t = Translation(
        [
            res[0] * (-center[0] / h + .5),
            res[1] * (-center[1] / h + .5)
        ]).compose_after(Scale((res[0] / h, res[1] / h))).pseudoinverse()

    # Upper left point
    ul = np.floor(t.apply([0, 0]))
    # Bottom right point
    br = np.ceil(t.apply(res).astype(np.int))

    # crop and rescale
    cimg, trans = img.warp_to_shape(
        br - ul, Translation(-(br - ul) / 2 + (br + ul) / 2), return_transform=True)

    c_scale = np.min(cimg.shape) / np.mean(res)
    new_img = cimg.rescale(1 / c_scale, order=order).resize(res, order=order)

    trans = trans.compose_after(Scale([c_scale, c_scale]))

    return new_img, trans

In [5]:
minsize = 40 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor

with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_detector(sess, './detection')

# 3D_84

In [ ]:
record_name = 'menpo.tfrecords'
record_save_path = Path('./3D84/')
image_load_path = Path('/media/jd4615/data3/alignment3D/3D84/data/helen/menpo')

def get_jpg_string(im):
    fp = BytesIO()
    mio.export_image(im, fp, extension='jpg')
    fp.seek(0)
    return fp.read()

def _int_feauture(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feauture(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feauture(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def face_iterator(image_load_path=image_load_path):

    for img in print_progress(mio.import_images(image_load_path, verbose=True)):
        image_path = img.path
        
        image_name = image_path.name

        status = 0
               
        pts = np.zeros((84,2))
        
        binary_mask = np.zeros(84)

        mask_index = np.arange(84)

        binary_mask[mask_index] = 1
        
        bbox = img.landmarks['LJSON'].lms.bounding_box()

        bounding_boxes = np.array([[bbox.points[0][1], bbox.points[0][0], bbox.points[2][1], bbox.points[2][0], 1.]])

        bounding_boxes, points = detect_face.box_regression(img.pixels_with_channels_at_back() * 255, onet, bounding_boxes, 0.001)
        
        bbox = PointCloud(bounding_boxes[0,[1,0,3,2]].reshape([2,2])).bounding_box()
        
        crop_img,_ = crop_image_bounding_box(img, bbox, [384., 384.], base=256./384., order=1)
                
        pts[mask_index] = crop_img.landmarks['LJSON'].lms.points

        yield image_name, crop_img, status, pts, binary_mask, mask_index

def generate(iterator,
             record_save_path=record_save_path,
             record_name=record_name,
             store_records=True):

    if store_records:
        writer = tf.python_io.TFRecordWriter(str(record_save_path/record_name))

    for image_name, img, status, pts, binary_mask, mask_index in iterator:

        if store_records:
            example = tf.train.Example(
                features=tf.train.Features(
                  feature={
                    # images
                    'image': _bytes_feauture(get_jpg_string(img)),
                    'height': _int_feauture(img.shape[0]),
                    'width': _int_feauture(img.shape[1]),
                    # landmarks
                    'n_landmarks': _int_feauture(len(pts)),
                    'gt_pts': _bytes_feauture(pts.astype(np.float32).tobytes()),
                    'gt_mask': _bytes_feauture(binary_mask.astype(np.float32).tobytes()),
                    'mask_index': _bytes_feauture(mask_index.astype(np.float32).tobytes()),
                    # status
                    'status': _int_feauture(status),
                        
            }))
            serialized = example.SerializeToString()
            writer.write(serialized)
    if store_records:
        writer.close()

generate(face_iterator(), record_save_path, record_name,  store_records=True)

# 2D_68_frontal 

In [ ]:
record_name = 'menpo_frontal.tfrecords'
record_save_path = Path('./2D68/')
image_load_path = Path('/media/jd4615/data3/alignment3D/3D84/data/menpo/refine/semifrontal')
landmarkNum = 68

def get_jpg_string(im):
    fp = BytesIO()
    mio.export_image(im, fp, extension='jpg')
    fp.seek(0)
    return fp.read()

def _int_feauture(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feauture(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feauture(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def face_iterator(image_load_path=image_load_path):

    for img in print_progress(mio.import_images(image_load_path, verbose=True)):
        image_path = img.path
        
        image_name = image_path.name

        status = 0
               
        pts = np.zeros((landmarkNum,2))
        
        binary_mask = np.zeros(landmarkNum)

        mask_index = np.arange(landmarkNum)

        binary_mask[mask_index] = 1
        
        bbox = img.landmarks['PTS'].lms.bounding_box()

        bounding_boxes = np.array([[bbox.points[0][1], bbox.points[0][0], bbox.points[2][1], bbox.points[2][0], 1.]])

        bounding_boxes, points = detect_face.box_regression(img.pixels_with_channels_at_back() * 255, onet, bounding_boxes, 0.000001)
        
        bbox = PointCloud(bounding_boxes[0,[1,0,3,2]].reshape([2,2])).bounding_box()
        
        crop_img,_ = crop_image_bounding_box(img, bbox, [384., 384.], base=256./384., order=1)
                
        pts[mask_index] = crop_img.landmarks['PTS'].lms.points

        yield image_name, crop_img, status, pts, binary_mask, mask_index

def generate(iterator,
             record_save_path=record_save_path,
             record_name=record_name,
             store_records=True):

    if store_records:
        writer = tf.python_io.TFRecordWriter(str(record_save_path/record_name))

    for image_name, img, status, pts, binary_mask, mask_index in iterator:

        if store_records:
            example = tf.train.Example(
                features=tf.train.Features(
                  feature={
                    # images
                    'image': _bytes_feauture(get_jpg_string(img)),
                    'height': _int_feauture(img.shape[0]),
                    'width': _int_feauture(img.shape[1]),
                    # landmarks
                    'n_landmarks': _int_feauture(len(pts)),
                    'gt_pts': _bytes_feauture(pts.astype(np.float32).tobytes()),
                    'gt_mask': _bytes_feauture(binary_mask.astype(np.float32).tobytes()),
                    'mask_index': _bytes_feauture(mask_index.astype(np.float32).tobytes()),
                    # status
                    'status': _int_feauture(status),
                        
            }))
            serialized = example.SerializeToString()
            writer.write(serialized)
    if store_records:
        writer.close()

generate(face_iterator(), record_save_path, record_name,  store_records=True)

# 2D_68_profile_right_39

In [ ]:
record_name = 'menpo_profile_right.tfrecords'
record_save_path = Path('./2D68/')
image_load_path = Path('/media/jd4615/data3/alignment3D/3D84/data/menpo/refine/profile_right')
landmarkNum = 68

def get_jpg_string(im):
    fp = BytesIO()
    mio.export_image(im, fp, extension='jpg')
    fp.seek(0)
    return fp.read()

def _int_feauture(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feauture(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feauture(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def face_iterator(image_load_path=image_load_path):

    for img in print_progress(mio.import_images(image_load_path, verbose=True)):
        image_path = img.path
        
        image_name = image_path.name
               
        pts = np.zeros((landmarkNum,2))
        
        binary_mask = np.zeros(landmarkNum)
        
        status = 1
        mask_index = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18, 19, 21, 22, 
                               28, 29, 30, 31, 34, 32, 39, 38, 37, 42, 41, 52, 51, 50, 49, 60, 59, 58, 63, 62, 61, 68, 67])-1

        binary_mask[mask_index] = 1
        
        bbox = img.landmarks['PTS'].lms.bounding_box()

        bounding_boxes = np.array([[bbox.points[0][1], bbox.points[0][0], bbox.points[2][1], bbox.points[2][0], 1.]])

        bounding_boxes, points = detect_face.box_regression(img.pixels_with_channels_at_back() * 255, onet, bounding_boxes, 0.000001)
        
        bbox = PointCloud(bounding_boxes[0,[1,0,3,2]].reshape([2,2])).bounding_box()
        
        crop_img,_ = crop_image_bounding_box(img, bbox, [384., 384.], base=256./384., order=1)
                
        pts[mask_index] = crop_img.landmarks['PTS'].lms.points
        
        mask_index = np.concatenate([mask_index,np.zeros(landmarkNum-len(mask_index))-1])

        yield image_name, crop_img, status, pts, binary_mask, mask_index

def generate(iterator,
             record_save_path=record_save_path,
             record_name=record_name,
             store_records=True):

    if store_records:
        writer = tf.python_io.TFRecordWriter(str(record_save_path/record_name))

    for image_name, img, status, pts, binary_mask, mask_index in iterator:

        if store_records:
            example = tf.train.Example(
                features=tf.train.Features(
                  feature={
                    # images
                    'image': _bytes_feauture(get_jpg_string(img)),
                    'height': _int_feauture(img.shape[0]),
                    'width': _int_feauture(img.shape[1]),
                    # landmarks
                    'n_landmarks': _int_feauture(len(pts)),
                    'gt_pts': _bytes_feauture(pts.astype(np.float32).tobytes()),
                    'gt_mask': _bytes_feauture(binary_mask.astype(np.float32).tobytes()),
                    'mask_index': _bytes_feauture(mask_index.astype(np.float32).tobytes()),
                    # status
                    'status': _int_feauture(status),
                        
            }))
            serialized = example.SerializeToString()
            writer.write(serialized)
    if store_records:
        writer.close()

generate(face_iterator(), record_save_path, record_name,  store_records=True)

# 2D_68_profile_left_39

In [ ]:
record_name = 'menpo_profile_left.tfrecords'
record_save_path = Path('./2D68/')
image_load_path = Path('/media/jd4615/data3/alignment3D/3D84/data/menpo/refine/profile_left')
landmarkNum = 68
def get_jpg_string(im):
    fp = BytesIO()
    mio.export_image(im, fp, extension='jpg')
    fp.seek(0)
    return fp.read()

def _int_feauture(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feauture(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feauture(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def face_iterator(image_load_path=image_load_path):

    for img in print_progress(mio.import_images(image_load_path, verbose=True)):
        image_path = img.path
        
        image_name = image_path.name
               
        pts = np.zeros((landmarkNum,2))
        
        binary_mask = np.zeros(landmarkNum)
        
        status = -1
        mask_index = np.array([17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 27, 26, 24, 23,
                               28, 29, 30, 31, 34, 36, 44, 45, 46, 47, 48, 52, 53, 54, 55, 56, 57, 58, 63, 64, 65, 66, 67])-1


        binary_mask[mask_index] = 1
        
        bbox = img.landmarks['PTS'].lms.bounding_box()

        bounding_boxes = np.array([[bbox.points[0][1], bbox.points[0][0], bbox.points[2][1], bbox.points[2][0], 1.]])

        bounding_boxes, points = detect_face.box_regression(img.pixels_with_channels_at_back() * 255, onet, bounding_boxes, 0.000001)
        
        bbox = PointCloud(bounding_boxes[0,[1,0,3,2]].reshape([2,2])).bounding_box()
        
        crop_img,_ = crop_image_bounding_box(img, bbox, [384., 384.], base=256./384., order=1)
                
        pts[mask_index] = crop_img.landmarks['PTS'].lms.points
        
        mask_index = np.concatenate([mask_index,np.zeros(landmarkNum-len(mask_index))-1])

        yield image_name, crop_img, status, pts, binary_mask, mask_index

def generate(iterator,
             record_save_path=record_save_path,
             record_name=record_name,
             store_records=True):

    if store_records:
        writer = tf.python_io.TFRecordWriter(str(record_save_path/record_name))

    for image_name, img, status, pts, binary_mask, mask_index in iterator:

        if store_records:
            example = tf.train.Example(
                features=tf.train.Features(
                  feature={
                    # images
                    'image': _bytes_feauture(get_jpg_string(img)),
                    'height': _int_feauture(img.shape[0]),
                    'width': _int_feauture(img.shape[1]),
                    # landmarks
                    'n_landmarks': _int_feauture(len(pts)),
                    'gt_pts': _bytes_feauture(pts.astype(np.float32).tobytes()),
                    'gt_mask': _bytes_feauture(binary_mask.astype(np.float32).tobytes()),
                    'mask_index': _bytes_feauture(mask_index.astype(np.float32).tobytes()),
                    # status
                    'status': _int_feauture(status),
                        
            }))
            serialized = example.SerializeToString()
            writer.write(serialized)
    if store_records:
        writer.close()

generate(face_iterator(), record_save_path, record_name,  store_records=True)